<img src=https://i.ytimg.com/vi/knxlmCVFAZI/maxresdefault.jpg>

In [1]:
#!pip3 install twilio

In [2]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# Armado de la URL

In [3]:
query = 'Salta'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=f3cf65ff7ae64bccad9210657220807&q=Salta&days=1&aqi=no&alerts=no'

In [4]:
response = requests.get(url_clima).json()

In [5]:
response

{'location': {'name': 'Salta',
  'region': 'Salta',
  'country': 'Argentina',
  'lat': -24.78,
  'lon': -65.42,
  'tz_id': 'America/Argentina/Salta',
  'localtime_epoch': 1708989791,
  'localtime': '2024-02-26 20:23'},
 'current': {'last_updated_epoch': 1708989300,
  'last_updated': '2024-02-26 20:15',
  'temp_c': 21.0,
  'temp_f': 69.8,
  'is_day': 0,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
   'code': 1003},
  'wind_mph': 4.3,
  'wind_kph': 6.8,
  'wind_degree': 250,
  'wind_dir': 'WSW',
  'pressure_mb': 1016.0,
  'pressure_in': 30.0,
  'precip_mm': 0.12,
  'precip_in': 0.0,
  'humidity': 94,
  'cloud': 75,
  'feelslike_c': 21.0,
  'feelslike_f': 69.8,
  'vis_km': 9.0,
  'vis_miles': 5.0,
  'uv': 1.0,
  'gust_mph': 8.7,
  'gust_kph': 14.0},
 'forecast': {'forecastday': [{'date': '2024-02-26',
    'date_epoch': 1708905600,
    'day': {'maxtemp_c': 22.5,
     'maxtemp_f': 72.4,
     'mintemp_c': 18.1,
     'mintemp_f': 64.5

In [6]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [7]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [8]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [16]:
response['forecast']['forecastday'][0]['hour'][1]

{'time_epoch': 1708920000,
 'time': '2024-02-26 01:00',
 'temp_c': 18.2,
 'temp_f': 64.8,
 'is_day': 0,
 'condition': {'text': 'Mist',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png',
  'code': 1030},
 'wind_mph': 1.3,
 'wind_kph': 2.2,
 'wind_degree': 258,
 'wind_dir': 'WSW',
 'pressure_mb': 1016.0,
 'pressure_in': 29.99,
 'precip_mm': 0.07,
 'precip_in': 0.0,
 'snow_cm': 0.0,
 'humidity': 95,
 'cloud': 53,
 'feelslike_c': 18.2,
 'feelslike_f': 64.8,
 'windchill_c': 18.2,
 'windchill_f': 64.8,
 'heatindex_c': 18.2,
 'heatindex_f': 64.8,
 'dewpoint_c': 17.5,
 'dewpoint_f': 63.5,
 'will_it_rain': 1,
 'chance_of_rain': 100,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 2.0,
 'vis_miles': 1.0,
 'gust_mph': 2.2,
 'gust_kph': 3.5,
 'uv': 1.0}

In [19]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0] #Fecha

'2024-02-26'

In [10]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]

'01'

In [11]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [12]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

0

# Dataframe

In [13]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(recoresponserd['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [14]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|███████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 114260.27it/s]


In [22]:
datos[0]

('2022-09-15', 0, 'Partly cloudy', 8.6, 0, 0)

In [20]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2022-09-15,0,Partly cloudy,8.6,0,0
1,2022-09-15,1,Patchy rain possible,8.4,0,60
2,2022-09-15,2,Partly cloudy,8.6,0,0
3,2022-09-15,3,Partly cloudy,8.5,0,0
4,2022-09-15,4,Partly cloudy,8.4,0,0
5,2022-09-15,5,Partly cloudy,8.1,0,0
6,2022-09-15,6,Partly cloudy,8.4,0,0
7,2022-09-15,7,Sunny,12.1,0,0
8,2022-09-15,8,Sunny,14.8,0,0
9,2022-09-15,9,Sunny,17.2,0,0


In [ ]:
# data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain
    
# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)

In [24]:
df[df['Lluvia']==1]

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
14,2022-09-15,14,Patchy rain possible,19.8,1,94
16,2022-09-15,16,Patchy rain possible,18.5,1,88


In [27]:
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 22)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [28]:
df['Fecha'][0]

'2022-09-15'

In [29]:
df_rain

,Condicion
Hora,
14,Patchy rain possible
16,Patchy rain possible


In [30]:
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico del tiempo hoy 2022-09-15 en Bogotá es : \n\n\n                  Condicion\nHora                      \n14    Patchy rain possible\n16    Patchy rain possible'

In [31]:
PHONE_NUMBER

'+1 614 401 3161'

# Mensaje SMS desde Twilio

In [32]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+573222007879'
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SMbc309ab2afc09ea1f87a7eb64fb8fdda


# Challenge 

* Extrae el valor del dolar en tu país y el top 10 de criptomonedas con su respectiva valoración
* Ahora envia un mensaje diarío a tu Whatsapp usando Twilio

**hint 💡** Investiga que API's gratuitas existen para consultar estos datos



<img src="WhatsApp Image 2022-09-13 at 9.12.18 AM.jpeg" width="200" height="200" />